In [1]:
import pandas as pd
import sys
import os
sys.path.insert(0, '../ghtesting')

from ghdatabase import GHDatabase
from ghrepo import GHRepo
from util import *
from macrogen import update_macro

In [2]:
db = GHDatabase('ecs260', 'webframework_repos', os.environ['CONNECTION_STRING'])

repos = [GHRepo(i) for i in db.get_repos()]

In [3]:
num_repos = len(repos)
update_macro('numrepos', f'{num_repos:,}')
print(f'Number of repos: {num_repos:,}')

Number of repos: 7,644


In [4]:
for wf, count in Counter([get_webframework(repo) for repo in repos]).items():
    key = 'numrepos' + wf.lower()
    update_macro(key, f'{count:,}')

In [5]:
top_badges = ['travisci', 'github', 'appveyorci', 'circleci', 'azure_pipelines']
def get_badge(badges):
    if len(badges) == 0:
        return 'None'
    for tb in top_badges:
        if tb in badges:
            return tb.replace('_', '')
    return 'other'

In [6]:
for badge, count in Counter([get_badge(repo.badges[0]) for repo in repos]).items():
    key = 'numrepos' + badge.lower()
    update_macro(key, f'{count:,}')
    
for badge, count in Counter([get_badge(repo.badges[0]) for repo in repos if get_webframework(repo) == 'Angular']).items():
    key = 'numreposangular' + badge.lower()
    update_macro(key, f'{count:,}')
    
for badge, count in Counter([get_badge(repo.badges[0]) for repo in repos if get_webframework(repo) == 'React']).items():
    key = 'numreposreact' + badge.lower()
    update_macro(key, f'{count:,}')
    
for badge, count in Counter([get_badge(repo.badges[0]) for repo in repos if get_webframework(repo) == 'Vue']).items():
    key = 'numreposvue' + badge.lower()
    update_macro(key, f'{count:,}')

In [7]:
cc = GHDatabase('ecs260', 'codecov_api', os.environ['CONNECTION_STRING'])
reports = list(cc.get_repos())

In [8]:
def get_webframework_by_repo_id(repo_id):
    return get_webframework([r for r in repos if r.name == repo_id][0])

In [9]:
def is_active_codecov_repos(report):
    return 'data' in report and 'repo' in report['data'] and report['data']['repo']['active']

In [10]:
update_macro('numreposcodecov', Counter([is_active_codecov_repos(report) for report in reports])[True])

In [11]:
update_macro('numreposangularcodecov', Counter([is_active_codecov_repos(report) for report in reports if get_webframework_by_repo_id(report['_id']) == 'Angular'])[True])
update_macro('numreposvuecodecov', Counter([is_active_codecov_repos(report) for report in reports if get_webframework_by_repo_id(report['_id']) == 'Vue'])[True])
update_macro('numreposreactcodecov', Counter([is_active_codecov_repos(report) for report in reports if get_webframework_by_repo_id(report['_id']) == 'React'])[True])